In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

In [2]:
griv=pd.read_csv(r"C:\Users\shubh\OneDrive\Desktop\Imarticus\NLP\greviance.csv")
griv=griv.sample(frac=1)

In [5]:
griv=griv.iloc[0:600000,:]

In [7]:
griv.shape

(600000, 18)

In [9]:
griv.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
59589,2023-10-26,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Their investigation did not fix an error on yo...,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TX,77036,NaN,NaN,Web,2023-10-26,In progress,Yes,NaN,7756289
2342331,2020-04-12,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,33027,NaN,Other,Web,2020-04-12,Closed with explanation,Yes,NaN,3603136
3151044,2012-12-19,Mortgage,Conventional adjustable mortgage (ARM),"Loan modification,collection,foreclosure",NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",CA,92592,NaN,NaN,Web,2012-12-20,Closed with explanation,Yes,Yes,213643
3521784,2013-08-28,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,PNC Bank N.A.,WI,53224,NaN,NaN,Web,2013-08-29,Closed with explanation,Yes,Yes,507710
96323,2023-10-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,Experian Information Solutions Inc.,AL,35208,NaN,NaN,Web,2023-10-20,In progress,Yes,NaN,7731761


In [15]:
griv.Product.unique()

array(['Credit reporting or other personal consumer reports',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Mortgage', 'Bank account or service',
       'Checking or savings account', 'Debt collection',
       'Credit card or prepaid card', 'Money transfers',
       'Credit reporting', 'Payday loan', 'Student loan', 'Credit card',
       'Consumer Loan', 'Vehicle loan or lease',
       'Payday loan, title loan, or personal loan',
       'Money transfer, virtual currency, or money service',
       'Payday loan, title loan, personal loan, or advance loan',
       'Prepaid card', 'Other financial service',
       'Debt or credit management', 'Virtual currency'], dtype=object)

In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [23]:
griv.Product=le.fit_transform(griv.Product)   #LABEL ENCODER ON single COLoUMN

In [25]:
griv['Issue']=griv['Issue'].astype(str)
griv['Sub-issue']=griv['Sub-issue'].astype(str)
griv['Consumer complaint narrative']=griv['Consumer complaint narrative'].astype(str)
griv['Company']=griv['Company'].astype(str)

In [27]:
griv['X']=(
    griv['Issue']+"  " +
    griv['Sub-issue']+ "  "+
    griv['Consumer complaint narrative']+"  "+
    griv['Company']
)

In [28]:
from nltk.corpus import stopwords
import string

import nltk
nltk.download('stopwords')

l1=list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shubh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
def text_process(mess):
    """
    1.remove Punctuation
    2.remove stopwords
    3.remove the list of clean textwords

    """
    nopunc=[char for char in mess if char not in string.punctuation]
    nopunc="".join(nopunc)

    return[word for word in nopunc.split() if word not in l1]

In [47]:
griv['X'].apply(text_process)

0          [Incorrect, information, report, Information, ...
1          [Incorrect, information, report, Account, info...
2          [Attempts, collect, debt, owed, Debt, already,...
3          [Improper, use, report, Credit, inquiries, rep...
4          [Incorrect, information, report, Information, ...
                                 ...                        
4280526    [Incorrect, information, report, Account, stat...
4280527    [Incorrect, information, report, Old, informat...
4280528    [Incorrect, information, report, Information, ...
4280529    [Incorrect, information, report, Information, ...
4280530    [Incorrect, information, report, Information, ...
Name: X, Length: 4280531, dtype: object

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer= CountVectorizer(analyzer=text_process).fit(griv["X"])

In [34]:
tdm=bow_transformer.transform(griv.X)

In [35]:
tdm.shape

(600000, 136184)

In [39]:
from sklearn.model_selection import train_test_split

In [41]:
x_train,x_test,y_train,y_test = train_test_split(tdm,griv.Product,test_size=0.2)

In [43]:

from sklearn.naive_bayes import MultinomialNB
mb=MultinomialNB()
mb.fit(x_train,y_train)

MultinomialNB()

In [45]:
pred_mb=mb.predict(x_test)

In [47]:
from sklearn.metrics import confusion_matrix,classification_report

In [49]:
confusion_matrix(y_test,pred_mb)

array([[ 2094,   265,     0,     2,    39,     1,     0,     2,     2,
            0,     7,     0,    14,     0,     0,     1,     0,     0,
            0,     2,     0],
       [    1,  5114,     0,     2,    62,     8,     9,    25,     2,
            0,    28,     0,    13,     0,     0,     1,     0,     0,
            0,     3,     0],
       [   56,    24,   529,     2,    15,     2,     0,     3,    23,
            0,     4,     0,    24,     0,     0,    24,     0,     0,
            9,   148,     0],
       [   20,    28,     0,  2079,   584,     1,    44,    81,    37,
            0,     5,     0,    22,     0,     0,     2,     0,     0,
            0,     1,     0],
       [    4,   281,     0,   173,  4846,     2,    63,   431,    34,
            0,     8,     0,    10,     0,     0,     3,     0,     0,
            5,     9,     0],
       [    0,     2,     0,     1,    45,  3162,    43,   588,    39,
            0,     0,     0,     9,     0,     0,     0,     0,     0

In [51]:
print(classification_report(y_test,pred_mb))

              precision    recall  f1-score   support

           0       0.91      0.86      0.88      2429
           1       0.84      0.97      0.90      5268
           2       0.98      0.61      0.75       863
           3       0.80      0.72      0.75      2904
           4       0.70      0.83      0.76      5869
           5       0.73      0.81      0.77      3897
           6       0.14      0.83      0.23      6004
           7       0.91      0.43      0.59     60830
           8       0.95      0.97      0.96     14500
           9       0.00      0.00      0.00         6
          10       0.84      0.72      0.78      1654
          11       0.92      0.07      0.13       160
          12       0.95      0.98      0.96     10814
          13       0.00      0.00      0.00        35
          14       0.93      0.09      0.16       147
          15       0.69      0.43      0.53       847
          16       0.00      0.00      0.00        34
          17       1.00    

# LOGISTIC REGRESSION

In [53]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(class_weight='balanced',)
logreg.fit(x_train,y_train)

LogisticRegression(class_weight='balanced')

In [54]:
pred_logreg=logreg.predict(x_test)
confusion_matrix(y_test,pred_logreg)

array([[ 2393,     8,     3,     1,     3,     1,     0,     1,     0,
            0,     9,     0,     5,     0,     1,     2,     0,     2,
            0,     0,     0],
       [   10,  5071,     3,     4,   110,     0,     4,    12,     0,
            1,    24,     0,     2,     0,     1,    12,     1,     1,
            1,    11,     0],
       [    1,     0,   822,     1,     0,     2,     0,     0,     2,
            0,     0,     0,     2,     0,    10,     5,     6,     0,
            5,     7,     0],
       [    1,     9,     3,  2561,   235,     1,    31,     9,     8,
            1,     8,     3,     4,    13,     4,     6,     3,     0,
            2,     2,     0],
       [    3,   133,     1,    92,  5117,     2,    37,   137,     5,
            0,    29,     0,     1,     0,     1,    33,    15,   198,
            5,    60,     0],
       [    0,     2,     1,     6,    11,  3774,     7,    84,     0,
            0,     1,     0,     3,     0,     1,     1,     1,     0

In [55]:
print(classification_report(y_test,pred_logreg))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2429
           1       0.96      0.96      0.96      5268
           2       0.90      0.95      0.93       863
           3       0.82      0.88      0.85      2904
           4       0.80      0.87      0.84      5869
           5       0.77      0.97      0.86      3897
           6       0.16      0.87      0.27      6004
           7       0.97      0.47      0.63     60830
           8       0.99      0.99      0.99     14500
           9       0.03      0.33      0.06         6
          10       0.88      0.83      0.86      1654
          11       0.46      0.80      0.58       160
          12       0.98      0.97      0.97     10814
          13       0.21      0.77      0.33        35
          14       0.55      0.84      0.67       147
          15       0.42      0.60      0.49       847
          16       0.03      0.62      0.06        34
          17       0.37    

# DECISION TREE

In [59]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)

DecisionTreeClassifier()

In [60]:
predict_dt=dt.predict(x_test)
confusion_matrix(y_test,predict_dt)

array([[ 2388,     0,     1,    13,     0,     0,     0,     1,     0,
            0,     8,     1,     8,     0,     0,     8,     0,     1,
            0,     0,     0],
       [    0,  5109,     0,     1,    98,     0,     3,    34,    13,
            0,     0,     0,     3,     0,     0,     2,     0,     2,
            3,     0,     0],
       [    1,     0,   824,     5,     0,     2,     0,     0,     0,
            0,     4,     2,     3,     0,     8,     8,     0,     1,
            5,     0,     0],
       [    2,     7,     1,  2483,   215,     1,    61,    28,     8,
            0,    19,     3,    60,     0,     0,    12,     0,     4,
            0,     0,     0],
       [    1,   134,     0,    65,  5244,     4,     6,   366,    17,
            0,     8,     1,     2,     0,     0,     8,     0,     6,
            3,     4,     0],
       [    1,     1,     0,     1,     2,  3823,     2,    64,     2,
            0,     0,     0,     0,     0,     0,     0,     0,     0

In [62]:
print(classification_report(y_test,predict_dt))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2429
           1       0.97      0.97      0.97      5268
           2       0.97      0.95      0.96       863
           3       0.92      0.86      0.89      2904
           4       0.90      0.89      0.90      5869
           5       0.98      0.98      0.98      3897
           6       0.73      0.24      0.37      6004
           7       0.92      0.98      0.95     60830
           8       0.99      1.00      1.00     14500
           9       0.14      0.17      0.15         6
          10       0.88      0.92      0.90      1654
          11       0.72      0.50      0.59       160
          12       0.98      0.98      0.98     10814
          13       0.79      0.54      0.64        35
          14       0.81      0.80      0.80       147
          15       0.80      0.79      0.80       847
          16       0.14      0.09      0.11        34
          17       0.83    